In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")
np.random.seed(203)
from tqdm import tqdm
import datetime
from collections import Counter
import re
from scipy import stats
from imblearn.under_sampling import ClusterCentroids

from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

from catboost import Pool, CatBoostClassifier

In [2]:
train_data = pd.read_csv('/Users/s0c02nj/Desktop/LTFS/train_aox2Jxw/train.csv')
test_data =  pd.read_csv('/Users/s0c02nj/Desktop/LTFS/test_bqCt9Pv.csv')
sub_data = pd.read_csv('/Users/s0c02nj/Desktop/LTFS/sample_submission_24jSKY6.csv')

In [3]:
col_names = list(train_data.columns)
col_names.remove('UniqueID')
col_names.remove('loan_default')

In [4]:
train_count=[]
for col in tqdm(col_names):
    val = len(train_data[col].unique())
    train_count.append((col,val))

100%|██████████| 39/39 [00:00<00:00, 297.59it/s]


In [5]:
x_train = train_data.drop('loan_default',axis=1)
y_def = train_data['loan_default']

In [6]:
x_train = x_train.drop('UniqueID',axis=1)
x_test = test_data.drop('UniqueID',axis=1)

In [7]:
x_comb = pd.concat([x_train,x_test],sort=False)

In [8]:
x_comb = x_comb.replace(np.nan, 'unknown', regex=True)

In [9]:
from datetime import timedelta, date
col = 'date'
x_comb['Date.of.Birth'] = pd.to_datetime(x_comb['Date.of.Birth'])
future = x_comb['Date.of.Birth'] > date(year=2050,month=1,day=1)
x_comb.loc[future, 'Date.of.Birth'] -= timedelta(days=365.25*100)

/Users/s0c02nj/anaconda2/envs/bert/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  after removing the cwd from sys.path.


In [10]:
from datetime import timedelta, date
col = 'date'
x_comb['DisbursalDate'] = pd.to_datetime(x_comb['DisbursalDate'])
future = x_comb['DisbursalDate'] > date(year=2050,month=1,day=1)
x_comb.loc[future, 'DisbursalDate'] -= timedelta(days=365.25*100)

/Users/s0c02nj/anaconda2/envs/bert/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  after removing the cwd from sys.path.


In [11]:
x_comb['age'] = 2018-x_comb['Date.of.Birth'].dt.year

In [12]:
x_comb['disbursion_wk'] = x_comb['DisbursalDate'].dt.week
x_comb['disbursion_mnth'] = x_comb['DisbursalDate'].dt.month
x_comb['disbursion_day'] = x_comb['DisbursalDate'].dt.day

In [13]:
x_comb['avg_acct_age_yr'] =  x_comb['AVERAGE.ACCT.AGE'].apply(lambda x:re.findall(r'\d+', x)[0]).astype(int)
x_comb['avg_acct_age_mon'] = x_comb['AVERAGE.ACCT.AGE'].apply(lambda x:re.findall(r'\d+', x)[1]).astype(int)

x_comb['cred_his_yr'] =  x_comb['CREDIT.HISTORY.LENGTH'].apply(lambda x:re.findall(r'\d+', x)[0]).astype(int)
x_comb['cred_his_mon'] = x_comb['CREDIT.HISTORY.LENGTH'].apply(lambda x:re.findall(r'\d+', x)[1]).astype(int)

In [14]:
x_comb['avg_acct_age_comb'] = x_comb['avg_acct_age_yr']*12.0 + x_comb['avg_acct_age_mon']
x_comb['cred_his_comb'] = x_comb['cred_his_yr']*12.0 + x_comb['cred_his_mon']

In [15]:
x_comb = x_comb.drop(['Date.of.Birth', 'DisbursalDate','AVERAGE.ACCT.AGE','CREDIT.HISTORY.LENGTH'], axis=1)

In [16]:
x_comb['DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS_cat'] = x_comb['DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS']

In [17]:
com_count={}
for col in tqdm(list(x_comb.columns)):
    val = len(x_comb[col].unique())
    com_count[col]=val

100%|██████████| 46/46 [00:00<00:00, 231.77it/s]


In [18]:
total_cols = list(x_comb.columns)

Categorizing Continuous based on bins

In [19]:
cat_cols = ['branch_id','supplier_id','manufacturer_id','Current_pincode_ID','Employment.Type',
           'State_ID','Employee_code_ID','Aadhar_flag','PAN_flag','VoterID_flag','Driving_flag',
           'Passport_flag','PERFORM_CNS.SCORE.DESCRIPTION',
           'disbursion_wk','disbursion_mnth','disbursion_day','PRI.NO.OF.ACCTS']

In [20]:
cont_cols = list(set(total_cols)-set(cat_cols))

In [21]:
for i,col in tqdm(enumerate(cont_cols)):
    counter = Counter(x_comb[col])
    x_comb[str(col)+'count'] = x_comb[col].apply(lambda x:counter[x])

29it [00:04,  6.58it/s]


In [22]:
cont_cols_new = list(set(list(x_comb.columns))-set(cat_cols))

1.Label Encoding Categorical Columns

In [23]:
for col in tqdm(cat_cols):
    le = LabelEncoder()
    x_comb[col] = le.fit_transform(x_comb[col])

100%|██████████| 17/17 [00:00<00:00, 52.34it/s]


In [24]:
train_x = x_comb[0:233154]
test_x = x_comb[233154:]

Removing TomekLinks

In [ ]:
# cc = ClusterCentroids(ratio={0: 172543})
# X_cc, y_cc = cc.fit_sample(train_x, y_def)


Target encoding for Categorical Variables

In [ ]:
# train_x4 = train_x.copy()
# train_x4['default'] = y_def

In [ ]:
# for i,col in tqdm(enumerate(cat_cols)):
#     mean = train_x4['default'].mean()
    
#     #Compute the number of values and the mean of each group
#     agg = train_x4.groupby(col)['default'].agg(['count', 'mean'])
#     counts = agg['count']
#     means = agg['mean']
    
#     #Compute the "smoothed" means
#     m=2000
#     smooth = (counts * means + m * mean) / (counts + m)
    
#     #Final_val
#     x_comb[str(col)+'count'] = x_comb[col].map(smooth)
    

In [ ]:
# for i,col in tqdm(enumerate(cat_cols)):
#     means = train_x1.groupby(col)['default'].mean()
#     x_comb[str(col)+'count'+'1'] = x_comb[col].map(means)

In [ ]:
# train_x = x_comb[0:233154]
# test_x = x_comb[233154:]

Outlier Treatment

In [53]:
q = train_x['disbursed_amount'].quantile(0.99)
train_x1 = train_x[train_x['disbursed_amount'] < q]
train_y = y_def[train_x['disbursed_amount'] < q]

In [54]:
q = train_x1['asset_cost'].quantile(0.99)
train_x2 = train_x1[train_x1['asset_cost'] < q]
train_y2 = train_y[train_x1['asset_cost'] < q]

In [55]:
q = train_x2['ltv'].quantile(0.99)
train_x3 = train_x2[train_x2['ltv'] < q]
train_y3 = train_y2[train_x2['ltv'] < q]

In [58]:
#cont_cols

In [60]:
train_x4.shape

(226029, 75)

In [64]:
x1_train, x1_val, y1_train, y1_val = train_test_split(train_x3, train_y3,test_size=0.2,random_state=0)

In [1]:
#let us make the catboost model, use_best_model params will make the model prevent overfitting
model = CatBoostClassifier(iterations=500, 
                           learning_rate=0.01, 
                           l2_leaf_reg=3.5, 
                           depth=8, 
                           rsm=0.98, 
                           loss_function= 'Logloss', 
                           eval_metric='F1',
                           use_best_model=True,
                           random_seed=42)

NameError: name 'CatBoostClassifier' is not defined

In [72]:
model.fit(x1_train,y1_train,cat_features=cat_cols, eval_set=(x1_val,y1_val))


0:	test: 0.5973276	best: 0.5973276 (0)	total: 294ms	remaining: 2m 26s
1:	test: 0.6191081	best: 0.6191081 (1)	total: 579ms	remaining: 2m 24s
2:	test: 0.6231867	best: 0.6231867 (2)	total: 878ms	remaining: 2m 25s
3:	test: 0.6226027	best: 0.6231867 (2)	total: 1.06s	remaining: 2m 11s
4:	test: 0.6258968	best: 0.6258968 (4)	total: 1.34s	remaining: 2m 13s
5:	test: 0.6271637	best: 0.6271637 (5)	total: 1.66s	remaining: 2m 16s
6:	test: 0.6299651	best: 0.6299651 (6)	total: 1.99s	remaining: 2m 19s
7:	test: 0.6302940	best: 0.6302940 (7)	total: 2.28s	remaining: 2m 20s
8:	test: 0.6327459	best: 0.6327459 (8)	total: 2.6s	remaining: 2m 22s
9:	test: 0.6333195	best: 0.6333195 (9)	total: 2.94s	remaining: 2m 23s
10:	test: 0.6327784	best: 0.6333195 (9)	total: 3.26s	remaining: 2m 24s
11:	test: 0.6326349	best: 0.6333195 (9)	total: 3.57s	remaining: 2m 25s
12:	test: 0.6322747	best: 0.6333195 (9)	total: 3.77s	remaining: 2m 21s
13:	test: 0.6325165	best: 0.6333195 (9)	total: 4.05s	remaining: 2m 20s
14:	test: 0.63357

115:	test: 0.6539259	best: 0.6539259 (115)	total: 39s	remaining: 2m 9s
116:	test: 0.6541741	best: 0.6541741 (116)	total: 39.3s	remaining: 2m 8s
117:	test: 0.6542912	best: 0.6542912 (117)	total: 39.7s	remaining: 2m 8s
118:	test: 0.6545747	best: 0.6545747 (118)	total: 40s	remaining: 2m 8s
119:	test: 0.6548217	best: 0.6548217 (119)	total: 40.4s	remaining: 2m 7s
120:	test: 0.6550975	best: 0.6550975 (120)	total: 40.7s	remaining: 2m 7s
121:	test: 0.6553227	best: 0.6553227 (121)	total: 41s	remaining: 2m 7s
122:	test: 0.6554797	best: 0.6554797 (122)	total: 41.3s	remaining: 2m 6s
123:	test: 0.6556681	best: 0.6556681 (123)	total: 41.6s	remaining: 2m 6s
124:	test: 0.6558776	best: 0.6558776 (124)	total: 41.9s	remaining: 2m 5s
125:	test: 0.6560888	best: 0.6560888 (125)	total: 42.2s	remaining: 2m 5s
126:	test: 0.6563827	best: 0.6563827 (126)	total: 42.5s	remaining: 2m 4s
127:	test: 0.6566617	best: 0.6566617 (127)	total: 42.8s	remaining: 2m 4s
128:	test: 0.6568932	best: 0.6568932 (128)	total: 43.2s	r

227:	test: 0.6652308	best: 0.6652308 (227)	total: 1m 19s	remaining: 1m 34s
228:	test: 0.6652970	best: 0.6652970 (228)	total: 1m 19s	remaining: 1m 34s
229:	test: 0.6653448	best: 0.6653448 (229)	total: 1m 19s	remaining: 1m 33s
230:	test: 0.6653834	best: 0.6653834 (230)	total: 1m 20s	remaining: 1m 33s
231:	test: 0.6654184	best: 0.6654184 (231)	total: 1m 20s	remaining: 1m 33s
232:	test: 0.6654983	best: 0.6654983 (232)	total: 1m 21s	remaining: 1m 33s
233:	test: 0.6655249	best: 0.6655249 (233)	total: 1m 21s	remaining: 1m 32s
234:	test: 0.6655669	best: 0.6655669 (234)	total: 1m 22s	remaining: 1m 32s
235:	test: 0.6656138	best: 0.6656138 (235)	total: 1m 22s	remaining: 1m 32s
236:	test: 0.6657065	best: 0.6657065 (236)	total: 1m 23s	remaining: 1m 32s
237:	test: 0.6657538	best: 0.6657538 (237)	total: 1m 23s	remaining: 1m 31s
238:	test: 0.6657936	best: 0.6657936 (238)	total: 1m 23s	remaining: 1m 31s
239:	test: 0.6658244	best: 0.6658244 (239)	total: 1m 24s	remaining: 1m 31s
240:	test: 0.6658318	best

337:	test: 0.6687065	best: 0.6687065 (337)	total: 1m 59s	remaining: 57.4s
338:	test: 0.6687157	best: 0.6687157 (338)	total: 2m	remaining: 57.1s
339:	test: 0.6687519	best: 0.6687519 (339)	total: 2m	remaining: 56.7s
340:	test: 0.6687566	best: 0.6687566 (340)	total: 2m	remaining: 56.3s
341:	test: 0.6687738	best: 0.6687738 (341)	total: 2m 1s	remaining: 56s
342:	test: 0.6687798	best: 0.6687798 (342)	total: 2m 1s	remaining: 55.6s
343:	test: 0.6687901	best: 0.6687901 (343)	total: 2m 1s	remaining: 55.2s
344:	test: 0.6688218	best: 0.6688218 (344)	total: 2m 2s	remaining: 54.9s
345:	test: 0.6688269	best: 0.6688269 (345)	total: 2m 2s	remaining: 54.5s
346:	test: 0.6688700	best: 0.6688700 (346)	total: 2m 2s	remaining: 54.1s
347:	test: 0.6688764	best: 0.6688764 (347)	total: 2m 3s	remaining: 53.8s
348:	test: 0.6689111	best: 0.6689111 (348)	total: 2m 3s	remaining: 53.4s
349:	test: 0.6689313	best: 0.6689313 (349)	total: 2m 3s	remaining: 53s
350:	test: 0.6689433	best: 0.6689433 (350)	total: 2m 4s	remaini

449:	test: 0.6707724	best: 0.6707724 (449)	total: 2m 38s	remaining: 17.6s
450:	test: 0.6707821	best: 0.6707821 (450)	total: 2m 38s	remaining: 17.2s
451:	test: 0.6708019	best: 0.6708019 (451)	total: 2m 38s	remaining: 16.9s
452:	test: 0.6708268	best: 0.6708268 (452)	total: 2m 39s	remaining: 16.5s
453:	test: 0.6708439	best: 0.6708439 (453)	total: 2m 39s	remaining: 16.2s
454:	test: 0.6708596	best: 0.6708596 (454)	total: 2m 39s	remaining: 15.8s
455:	test: 0.6708713	best: 0.6708713 (455)	total: 2m 40s	remaining: 15.5s
456:	test: 0.6708960	best: 0.6708960 (456)	total: 2m 40s	remaining: 15.1s
457:	test: 0.6709177	best: 0.6709177 (457)	total: 2m 40s	remaining: 14.7s
458:	test: 0.6709382	best: 0.6709382 (458)	total: 2m 41s	remaining: 14.4s
459:	test: 0.6709482	best: 0.6709482 (459)	total: 2m 41s	remaining: 14s
460:	test: 0.6709627	best: 0.6709627 (460)	total: 2m 41s	remaining: 13.7s
461:	test: 0.6709895	best: 0.6709895 (461)	total: 2m 42s	remaining: 13.3s
462:	test: 0.6710018	best: 0.6710018 (46

In [73]:
#last let us make the submission,note that you have to make the pred to be int!
pred = model.predict_proba(test_x)
preds= pred[:,1]

In [74]:
sub_data['loan_default'] = preds

In [75]:
sub_data.to_csv('/Users/s0c02nj/Desktop/LTFS/Submission18.csv',index=False)

In [ ]:
0.672739